In [4]:
#import bqplot
import datetime
import dateutil.parser
import ee
#import ipywidgets
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets
import matplotlib.pyplot as plt
import ee.mapclient

In [5]:
# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)
ee.Initialize()

# all geometry per atoll is found in our atoll database
countryCode = 'MH'
OceanBasin = 'P'

atoll_name = 'Erikub'

geometry = [[169.8321533203125, 9.199715262283302],
                  [170.057373046875, 8.923060419880334],
                  [170.2001953125, 9.05870215639214],
                  [170.0189208984375, 9.291885812275517],
                  [169.8651123046875, 9.286464684304082]]
Region = ee.Geometry.Polygon([geometry]);

In [6]:
#OLD SOURCE OF DATA
Region = ee.Geometry.Polygon([geometry]);
studyArea = ee.FeatureCollection([ee.Feature(Region)])
startYear = 2015
endYear = 2018

#pp.pprint(studyArea.getInfo())

def getLandsat(startTime, endTime, studyArea, LandsatSatellite):
    st = str(startTime) + "-01-01"
    et = str(endTime) + "-12-31"
    if LandsatSatellite[1] == '8':
        lsat = 'LANDSAT/LC08/C01/T1_TOA'
        bOrder=['B2','B3','B4','B5','B6','B11','B7']
    elif LandsatSatellite[1] == '7':
        lsat = 'LANDSAT/LE07/C01/T1_TOA'
        bOrder=['B1','B2','B3','B4', 'B5','B6_VCID_2','B7']
    elif LandsatSatellite[1] == '5':
        lsat = 'LANDSAT/LT05/C01/T1_TOA'
        bOrder=['B1','B2','B3','B4', 'B5','B6','B7']
    elif LandsatSatellite[1] == '4':
        lsat = 'LANDSAT/LT04/C01/T1_TOA'
        bOrder=['B1','B2','B3','B4', 'B5','B6','B7']
    images = ee.ImageCollection(lsat).filterDate(st,et).filterBounds(studyArea).select(bOrder).filter(ee.Filter.eq('WRS_PATH',83))
    return images, bOrder

def maskCloudsImage(img):
    cloudThresh = 50
    cs = ee.Algorithms.Landsat.simpleCloudScore(img).select(['cloud']).gt(cloudThresh)
    cloudless = img.mask(img.mask().And(cs.Not()))
    return cloudless
    
def maskCloudsCollection(InImages,bOrder):
    outImages = InImages.map(maskCloudsImage)
    bandNames = ee.List(['blue','green','red','nir','swir1','temp','swir2'])
    outImages = outImages.select(bOrder,bandNames) 
    return outImages

image8, bOrder8 = getLandsat(startYear,endYear,studyArea,'L8')
image8 = maskCloudsCollection(image8,bOrder8)
image7, bOrder7 = getLandsat(startYear,endYear,studyArea,'L7')
image7 = maskCloudsCollection(image7,bOrder7)

image78 = image7.merge(image8);
pp.pprint(image78.getInfo())

{'bands': [],
 'features': [{'bands': [{...}, {...}, {...}, {...}, {...}, {...}, {...}],
               'id': 'LANDSAT/LC08/C01/T1_TOA/LC08_083054_20150117',
               'properties': {'BPF_NAME_OLI': 'LO8BPF20150117223858_20150117232309.01',
                              'BPF_NAME_TIRS': 'LT8BPF20150106000000_20150131235959.03',
                              'CLOUD_COVER': 19.639999389648438,
                              'CLOUD_COVER_LAND': 37.83000183105469,
                              'COLLECTION_CATEGORY': 'T1',
                              'COLLECTION_NUMBER': 1,
                              'CPF_NAME': 'LC08CPF_20150101_20150302_01.01',
                              'DATA_TYPE': 'L1TP',
                              'DATE_ACQUIRED': '2015-01-17',
                              'DATUM': 'WGS84',
                              'EARTH_SUN_DISTANCE': 0.983828604221344,
                              'ELEVATION_SOURCE': 'GLS2000',
                              'ELLIPSOID': 'WGS84

                              'REFLECTANCE_MINIMUM_BAND_7': -0.09997999668121338,
                              'REFLECTANCE_MINIMUM_BAND_8': -0.09997999668121338,
                              'REFLECTANCE_MINIMUM_BAND_9': -0.09997999668121338,
                              'REFLECTANCE_MULT_BAND_1': 1.9999999494757503e-05,
                              'REFLECTANCE_MULT_BAND_2': 1.9999999494757503e-05,
                              'REFLECTANCE_MULT_BAND_3': 1.9999999494757503e-05,
                              'REFLECTANCE_MULT_BAND_4': 1.9999999494757503e-05,
                              'REFLECTANCE_MULT_BAND_5': 1.9999999494757503e-05,
                              'REFLECTANCE_MULT_BAND_6': 1.9999999494757503e-05,
                              'REFLECTANCE_MULT_BAND_7': 1.9999999494757503e-05,
                              'REFLECTANCE_MULT_BAND_8': 1.9999999494757503e-05,
                              'REFLECTANCE_MULT_BAND_9': 1.9999999494757503e-05,
                         

                              'RADIANCE_MULT_BAND_11': 0.00033420001273043454,
                              'RADIANCE_MULT_BAND_2': 0.013042000122368336,
                              'RADIANCE_MULT_BAND_3': 0.012017999775707722,
                              'RADIANCE_MULT_BAND_4': 0.010134000331163406,
                              'RADIANCE_MULT_BAND_5': 0.0062015000730752945,
                              'RADIANCE_MULT_BAND_6': 0.0015422999858856201,
                              'RADIANCE_MULT_BAND_7': 0.0005198299768380821,
                              'RADIANCE_MULT_BAND_8': 0.011469000019133091,
                              'RADIANCE_MULT_BAND_9': 0.0024236999452114105,
                              'REFLECTANCE_ADD_BAND_1': -0.10000000149011612,
                              'REFLECTANCE_ADD_BAND_2': -0.10000000149011612,
                              'REFLECTANCE_ADD_BAND_3': -0.10000000149011612,
                              'REFLECTANCE_ADD_BAND_4': -0.1000000014901161

                              'TARGET_WRS_PATH': 83,
                              'TARGET_WRS_ROW': 54,
                              'THERMAL_LINES': 7741,
                              'THERMAL_SAMPLES': 7581,
                              'TIRS_SSM_MODEL': 'FINAL',
                              'TIRS_SSM_POSITION_STATUS': 'ESTIMATED',
                              'TIRS_STRAY_LIGHT_CORRECTION_SOURCE': 'TIRS',
                              'TRUNCATION_OLI': 'UPPER',
                              'UTM_ZONE': 59,
                              'WRS_PATH': 83,
                              'WRS_ROW': 54,
                              'google:registration_count': 51,
                              'google:registration_offset_x': 0,
                              'google:registration_offset_y': 0.00014361203648149967,
                              'google:registration_ratio': 1,
                              'system:asset_size': 1123841517,
                              'system:footprint': 

                              'SATURATION_BAND_1': 'N',
                              'SATURATION_BAND_2': 'Y',
                              'SATURATION_BAND_3': 'N',
                              'SATURATION_BAND_4': 'Y',
                              'SATURATION_BAND_5': 'Y',
                              'SATURATION_BAND_6': 'N',
                              'SATURATION_BAND_7': 'N',
                              'SATURATION_BAND_8': 'N',
                              'SATURATION_BAND_9': 'N',
                              'SCENE_CENTER_TIME': '22:55:08.2584519Z',
                              'SENSOR_ID': 'OLI_TIRS',
                              'SPACECRAFT_ID': 'LANDSAT_8',
                              'STATION_ID': 'LGN',
                              'SUN_AZIMUTH': 78.6776351928711,
                              'SUN_ELEVATION': 63.60167694091797,
                              'TARGET_WRS_PATH': 83,
                              'TARGET_WRS_ROW': 54,
                        

                              'MAP_PROJECTION': 'UTM',
                              'NADIR_OFFNADIR': 'NADIR',
                              'ORIENTATION': 'NORTH_UP',
                              'PANCHROMATIC_LINES': 15501,
                              'PANCHROMATIC_SAMPLES': 15181,
                              'PROCESSING_SOFTWARE_VERSION': 'LPGS_13.1.0',
                              'RADIANCE_ADD_BAND_1': -60.808631896972656,
                              'RADIANCE_ADD_BAND_10': 0.10000000149011612,
                              'RADIANCE_ADD_BAND_11': 0.10000000149011612,
                              'RADIANCE_ADD_BAND_2': -62.26877975463867,
                              'RADIANCE_ADD_BAND_3': -57.3801383972168,
                              'RADIANCE_ADD_BAND_4': -48.38616943359375,
                              'RADIANCE_ADD_BAND_5': -29.609920501708984,
                              'RADIANCE_ADD_BAND_6': -7.363719940185547,
                              'RADIANCE_ADD_BA

In [7]:
count = image78.reduce(ee.Reducer.median().combine(ee.Reducer.count(), "null", True))
countO = count.select([1],['count']);
countO = countO.float()
pp.pprint(countO.getInfo())

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'count'}],
 'type': 'Image'}


In [8]:
maskBand = countO.clip(Region)
pp.pprint(maskBand.getInfo())

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'dimensions': [2, 2],
            'id': 'count',
            'origin': [169, 8]}],
 'properties': {'system:footprint': {'coordinates': [[...]],
                                     'type': 'Polygon'}},
 'type': 'Image'}


In [9]:
def composite(inImages,countBand,maskedBand):
    bandNames = ee.List(['blue','green','red','nir','swir1','temp','swir2'])
    bandNumbers = [0,1,2,3,4,5,6]
    reducer = ee.Reducer.percentile([50]) #Reducer for compositing
    outImage = inImages.reduce(reducer).select(bandNumbers,bandNames)
    outImage2 = outImage.addBands(countBand)
    outImage3 = outImage2.addBands(maskedBand)
    return outImage3

composite78 = composite(image78,countO,maskBand)
pp.pprint(composite78.getInfo())

{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'blue'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'green'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'red'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'nir'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'float', 'type': 'PixelType'},
            'id': 'swir1'},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'pr

In [10]:
pp.pprint(composite78)


In [9]:
compositeName = OceanBasin + '_' + countryCode + '_' + atoll_name + '50c50mCountClip3_2015_2018_new'

task = ee.batch.Export.image.toDrive(image = composite78, 
                                     region = geometry,
                                     description = 'test',
                                     scale = 30,
                                     fileNamePrefix = compositeName,
                                     folder='AtollComposites_ACO') 
                                    

task.start()

In [10]:
print(compositeName)

P_MH_Erikub50c50mCountClip3_2015_2018_new


In [26]:
task.status()

{'state': 'COMPLETED',
 'description': 'test',
 'creation_timestamp_ms': 1643127126691,
 'update_timestamp_ms': 1643127344491,
 'start_timestamp_ms': 1643127148140,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/'],
 'attempt': 1,
 'id': 'CYBWKIEPNQMALZLLANMYSFVP',
 'name': 'projects/earthengine-legacy/operations/CYBWKIEPNQMALZLLANMYSFVP'}